In [ ]:
#Load modules
import sys
import os
import json
import csv
from datetime import date

In [ ]:
#Read
inputFile = open('schemas.csv', 'r')
csv = csv.DictReader(inputFile)
schemas = []
for row in csv:
    schemas.append(row)


In [ ]:
#Group by category

uncapitalize = lambda s: s[:1].lower() + s[1:] if s else ''

measurementTypes = {}
measurementTypes['categories'] = {}
for schema in schemas:
    if schema['category'] not in measurementTypes['categories']:
        measurementTypes['categories'][schema['category']] = []
    measurementTypes['categories'][schema['category']].append(schema)
    
#group by measurement type
grouped = {}
for category in measurementTypes['categories']:
    if category not in grouped:
        grouped[category] = {}
    for entry in measurementTypes['categories'][category]:
        if entry['measurementType'] not in grouped[category]:
           grouped[category][entry['measurementType']] = {}
        grouped[category][entry['measurementType']][entry['subtype']] = {
                #"measurementType":  entry['measurementTypeID'],
                "measurementType":  "akenza/" + uncapitalize(entry['category']) + "/" + entry['measurementType'] + "/" + entry['subtype'],
                "title":            entry['title'],
                "description":      entry['description'],
                "type":             entry['datatype']
            }
        if entry['unit'] != '':
            grouped[category][entry['measurementType']][entry['subtype']]['unit'] = entry['unit']
        if entry['minimum'] != '':
            if entry['minimum'].isdigit():
                grouped[category][entry['measurementType']][entry['subtype']]['minimum'] = int(entry['minimum'])
            else:
                grouped[category][entry['measurementType']][entry['subtype']]['minimum'] = float(entry['minimum'])
        if entry['maximum'] != '':
            if entry['maximum'].isdigit():
                grouped[category][entry['measurementType']][entry['subtype']]['maximum'] = int(entry['maximum'])
            else:
                grouped[category][entry['measurementType']][entry['subtype']]['maximum'] = float(entry['maximum'])
        

    grouped[category]['categoryDescription'] = entry['categoryDescription']
    #grouped[category]['id'] = entry['id']
    grouped[category]['id'] = 'https://raw.githubusercontent.com/akenza-io/device-type-library/main/data-models/' + category.lower() + '/schema.json',

In [ ]:
#Write files

for category in grouped:
    os.makedirs(os.path.dirname('./' + category.lower() + '/'), exist_ok=True)

    with open('./' + category.lower() + '/schema.json', 'w') as outputFile:
        fileTemplate = {
            '$schema': 'http://json-schema.org/draft-07/schema#',
            '$id': 'https://raw.githubusercontent.com/akenza-io/device-type-library/main/data-models/' + uncapitalize(category) + '/schema.json',
            'title': category,
            'description': grouped[category]['categoryDescription'],
            'type': 'object',
        }
        del grouped[category]['categoryDescription']
        del grouped[category]['id']
        fileTemplate['$defs'] = grouped[category]

        json.dump(fileTemplate, outputFile, ensure_ascii=False, indent=4)

